#### Randomly generate Environmental data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Parameters
num_silos = 20
num_grain_temp_per_silo = 3
start_date = datetime(2024, 4, 29)
end_date = datetime(2024, 6, 30)
time_intervals = [("08:00", "morning"), ("14:00", "afternoon")]
hour_intervals = [(hour, 0) for hour in range(8, 18)]

# Calculate the number of days
num_days = (end_date - start_date).days + 1

# Generate data
data = []

for silo in range(1, num_silos + 1):
    location = f"Silo{silo}"
    temp_device = f"Temp{silo}"
    humd_device = f"Humd{silo}"
    in_temp_device = f"InTemp{silo}"
    
    for day in range(num_days):  # Generate data for each day in the date range
        current_date = start_date + timedelta(days=day)
        for time, period in time_intervals:
            timestamp = current_date.strftime(f"%Y-%m-%d {time}:00")
            temp_value = round(np.random.uniform(8, 25), 1)
            humd_value = round(np.random.uniform(5, 35), 1)
            in_temp_value = round(np.random.uniform(6, 25), 1)
            
            data.append([temp_device, timestamp, location, "temperature", temp_value, "celsius"])
            data.append([humd_device, timestamp, location, "humidity", humd_value, "percentage"])
            data.append([in_temp_device, timestamp, location, "intemperature", in_temp_value, "celsius"])
        
        # Generate grain temperature data every day from 8 AM to 5 PM
        for hour, minute in hour_intervals:
            grain_temp_time = current_date.replace(hour=hour, minute=minute, second=0).strftime("%Y-%m-%d %H:%M:%S")
                
            for grain_temp_num in range(1, num_grain_temp_per_silo + 1):
                grain_temp_device = f"GrainTemp{(silo-1)*num_grain_temp_per_silo + grain_temp_num}"
                grain_temp_value = round(np.random.uniform(1, 15), 1)
                    
                data.append([grain_temp_device, grain_temp_time, location, "graintemperature", grain_temp_value, "celsius"])

# Create DataFrame
columns = ["IoTDeviceID", "timestamp", "Location", "Type", "Value", "Unit"]
df = pd.DataFrame(data, columns=columns)

# Save to CSV
csv_path = "Silo_iot_date.csv"
df.to_csv(csv_path, index=False)


##### reshape the environmental data csv

In [2]:
# Load the CSV file
file_path = 'Silo_iot_date.csv'
df = pd.read_csv(file_path)

# Initialize a dictionary to hold the reshaped data
reshaped_data = []

# Iterate over each unique combination of Location and timestamp
for (location, timestamp), group in df.groupby(['Location', 'timestamp']):
    row = {'Location': location, 'timestamp': timestamp}
    for _, record in group.iterrows():
        if record['Type'] == 'temperature':
            row['Temperature'] = record['Value']
            row['Temperature_IoTDeviceID'] = record['IoTDeviceID']
        elif record['Type'] == 'humidity':
            row['Humidity'] = record['Value']
            row['Humidity_IoTDeviceID'] = record['IoTDeviceID']
        elif record['Type'] == 'intemperature':
            row['InTemperature'] = record['Value']
            row['InTemperature_IoTDeviceID'] = record['IoTDeviceID']
        elif record['Type'] == 'graintemperature':
            if 'GrainTemperature1' not in row:
                row['GrainTemperature1'] = record['Value']
                row['GrainTemperature1_IoTDeviceID'] = record['IoTDeviceID']
            elif 'GrainTemperature2' not in row:
                row['GrainTemperature2'] = record['Value']
                row['GrainTemperature2_IoTDeviceID'] = record['IoTDeviceID']
            else:
                row['GrainTemperature3'] = record['Value']
                row['GrainTemperature3_IoTDeviceID'] = record['IoTDeviceID']
    reshaped_data.append(row)

# Convert the list of dictionaries to a dataframe
reshaped_df = pd.DataFrame(reshaped_data)

# Replace NaN values with 'null' for IoTDeviceID columns and 0.0 for measurement columns
reshaped_df.fillna({
    'Temperature': 0.0,
    'Temperature_IoTDeviceID': 'null',
    'Humidity': 0.0,
    'Humidity_IoTDeviceID': 'null',
    'InTemperature': 0.0,
    'InTemperature_IoTDeviceID': 'null',
    'GrainTemperature1': 0.0,
    'GrainTemperature1_IoTDeviceID': 'null',
    'GrainTemperature2': 0.0,
    'GrainTemperature2_IoTDeviceID': 'null',
    'GrainTemperature3': 0.0,
    'GrainTemperature3_IoTDeviceID': 'null'
}, inplace=True)

reshaped_df.insert(reshaped_df.columns.get_loc("GrainTemperature3_IoTDeviceID") + 1, '', None)

# Save the reshaped dataframe to a new CSV file
reshaped_df.to_csv('Reshaped_Silo_IoT_Data.csv', index=False)

reshaped_df.head()


,Location,timestamp,Temperature,Temperature_IoTDeviceID,Humidity,Humidity_IoTDeviceID,InTemperature,InTemperature_IoTDeviceID,GrainTemperature1,GrainTemperature1_IoTDeviceID,GrainTemperature2,GrainTemperature2_IoTDeviceID,GrainTemperature3,GrainTemperature3_IoTDeviceID,
0,Silo1,2024-04-29 08:00:00,16.2,Temp1,28.5,Humd1,20.5,InTemp1,6.7,GrainTemp1,12.7,GrainTemp2,11.4,GrainTemp3,None
1,Silo1,2024-04-29 09:00:00,0.0,null,0.0,null,0.0,null,13.9,GrainTemp1,14.5,GrainTemp2,11.1,GrainTemp3,None
2,Silo1,2024-04-29 10:00:00,0.0,null,0.0,null,0.0,null,12.7,GrainTemp1,4.9,GrainTemp2,3.4,GrainTemp3,None
3,Silo1,2024-04-29 11:00:00,0.0,null,0.0,null,0.0,null,6.4,GrainTemp1,1.1,GrainTemp2,3.4,GrainTemp3,None
4,Silo1,2024-04-29 12:00:00,0.0,null,0.0,null,0.0,null,12.6,GrainTemp1,10.5,GrainTemp2,3.6,GrainTemp3,None


In [3]:
reshaped_df.shape[0]

12600

#### Randomly generate weightsensor data

In [4]:
import pandas as pd

# Parameters
num_devices = 20
device_prefix = "WS"
location_prefix = "weighbridge"

# Generate data
data = []
for i in range(1, num_devices + 1):
    device_id = f"{device_prefix}{i:02d}"
    location = f"{location_prefix}{i}"
    device_type = "weight sensor"
    data.append([device_id, location, device_type])

# Create DataFrame
columns = ["IoTDeviceID", "Location", "Type"]
df = pd.DataFrame(data, columns=columns)

# Save to CSV
csv_path = "weight_sensor_data.csv"
df.to_csv(csv_path, index=False)

# print(f"CSV file has been saved as {csv_path}")


#### Randomly generate empty truck weight history

In [5]:
input_csv_path = './object_initialiser_iot/Object_Truck.csv'
df1 = pd.read_csv(input_csv_path, delimiter=';')
# Extract the fourth column (index 3, as indexing starts from 0)
df_extracted = df1.iloc[:, [3]]

In [6]:
def random_weight():
    return np.round(np.random.uniform(11000.0, 16500.0), 1)

df_extracted['manufactured_weight'] = df_extracted.index.to_series().apply(lambda x: random_weight())
df_extracted['last_but_one_empty_weight'] = df_extracted.index.to_series().apply(lambda x: random_weight())
df_extracted['last_weighed_empty_weight'] = df_extracted.index.to_series().apply(lambda x: random_weight())

C:\Users\n9810200\AppData\Local\Temp/ipykernel_30296/2304647793.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['manufactured_weight'] = df_extracted.index.to_series().apply(lambda x: random_weight())
C:\Users\n9810200\AppData\Local\Temp/ipykernel_30296/2304647793.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extracted['last_but_one_empty_weight'] = df_extracted.index.to_series().apply(lambda x: random_weight())
C:\Users\n9810200\AppData\Local\Temp/ipykernel_30296/2304647793.py:6: 

In [7]:
# Save the new dataframe to a CSV file
output_csv_path = 'empty_weight_history.csv'
df_extracted.to_csv(output_csv_path, index=False)